# Solana Manager Tools

Este notebook contiene herramientas completas para gestionar wallets, transferencias y utilidades de Solana.

### Importaciones y Configuración

In [ ]:
from solana_manager import SolanaWalletManager, SolanaAccountInfo, SolanaTransfer, SolanaUtils
import os

In [ ]:
# Configuración de red
NETWORK = "mainnet-beta"
RPC_URL = "https://api.mainnet-beta.solana.com"

## 1. Gestión de Wallets

Herramientas para crear, cargar y gestionar wallets de Solana.

### 1.1 Crear Nueva Wallet

In [ ]:
# Crear nueva wallet y guardarla en archivo
async def create_wallet():
    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        wallet_filename = await wallet_manager.create_wallet_file(filename=None)

        if wallet_filename:
            print(f"\n📁 Wallet guardada en: {wallet_filename}")
            print(f"📍 Dirección: {wallet_manager.get_address()}")
            print("⚠️  IMPORTANTE: Guarda este archivo de forma segura")

await create_wallet()

### 1.2 Cargar Wallet Existente

In [ ]:
# Cargar wallet desde archivo (cambia por tu archivo)
async def load_wallet():
    WALLET_FILE = "wallets/mi_wallet_20250611_192531.json"

    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        if os.path.exists(WALLET_FILE):
            success = await wallet_manager.load_wallet(WALLET_FILE)
            if success:
                print(f"✅ Wallet cargada exitosamente")
                print(f"📍 Dirección: {wallet_manager.get_address() or 'No se pudo obtener la dirección'}")
            else:
                print("❌ Error cargando wallet")
        else:
            print(f"❌ Archivo {WALLET_FILE} no encontrado")
            print("💡 Ejecuta la celda anterior para crear una nueva wallet")

await load_wallet()

### 1.3 Cargar Wallet desde Clave Privada

In [ ]:
async def load_wallet_from_private_key():
    # Cargar wallet desde clave privada
    PRIVATE_KEY = "YOUR PRIVATE KEY HERE"

    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        keypair = await wallet_manager.load_wallet_from_private_key(PRIVATE_KEY)
        if keypair:
            print(f"✅ Wallet cargada desde clave privada")
            print(f"📍 Dirección: {wallet_manager.get_address()}")

        print("💡 Descomenta el código de arriba y agrega tu clave privada para usar esta función")

await load_wallet_from_private_key()

## 2. Información de Cuentas

Consultar balances, tokens y estado de cuentas.

### 2.1 Balance Completo de Wallet

In [ ]:
async def get_balance_info():
    WALLET_FILE = "wallets/wallet_pumpportal.json"

    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        await wallet_manager.load_wallet(WALLET_FILE)
        if wallet_manager.is_wallet_loaded():
            address = wallet_manager.get_address()

            async with SolanaAccountInfo(network="mainnet-beta") as account_info:
                balance_info = await account_info.get_balance_info(address)

                print(f"\n💰 BALANCE COMPLETO")
                print(f"📍 Wallet: {address}")
                print(f"💎 SOL: {balance_info['sol_balance']:.9f}")
                print(f"💵 Valor USD: ${balance_info['usd_value']:.2f}")
                print(f"📊 Precio SOL: ${balance_info['sol_price_usd']:.2f}")
        else:
            print("❌ No hay wallet cargada. Ejecuta las celdas de la sección 1 primero.")

await get_balance_info()

### 2.2 Tokens SPL en la Wallet

In [ ]:
async def get_token_accounts():
    WALLET_FILE = "wallets/wallet_pumpportal.json"

    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        await wallet_manager.load_wallet(WALLET_FILE)

        if not wallet_manager.is_wallet_loaded():
            print("❌ No hay wallet cargada")
            return

        async with SolanaAccountInfo(network="mainnet-beta") as account_info:
            address = wallet_manager.get_address()
            token_accounts = await account_info.get_token_accounts(address)

            if not token_accounts:
                print("❌ No se encontraron tokens SPL en esta wallet")
                return

            print(f"\n🪙 TOKENS SPL ENCONTRADOS: {len(token_accounts)}")
            print("=" * 60)

            i = 0
            for i, token in enumerate(token_accounts, 1):
                    print(f"\n{i}. Token:")
                    print(f"   📍 Mint: {token['mint']}")
                    print(f"   💰 Balance: {token['balance']:,.{min(token['decimals'], 6)}f}")
                    print(f"   🔢 Decimales: {token['decimals']}")
                    print(f"   🏦 Cuenta: {token['account_address']}")

            if i == 0:
                print("📭 No se encontraron tokens SPL en esta wallet")

await get_token_accounts()

### 2.3 Información Detallada de Cuenta

In [ ]:
async def get_account_info():
    WALLET_FILE = "wallets/wallet_pumpfun.json"

    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        await wallet_manager.load_wallet(WALLET_FILE)
        if not wallet_manager.is_wallet_loaded():
            print("❌ No hay wallet cargada")
            return

        async with SolanaAccountInfo(network="mainnet-beta") as account_info:
            address = wallet_manager.get_address()
            account_details = await account_info.get_account_info(address)

            print(f"\n🔍 INFORMACIÓN DETALLADA DE CUENTA")
            print("=" * 50)
            print(f"📍 Dirección: {account_details['address']}")
            print(f"💰 Balance SOL: {account_details['sol_balance']:.9f}")
            print(f"🏦 Lamports: {account_details['lamports']:,}")
            print(f"✅ Existe: {account_details['exists']}")
            print(f"🌐 Red: {account_details['network']}")

            if account_details.get('executable') is not None:
                print(f"⚙️ Ejecutable: {account_details['executable']}")
                print(f"👤 Owner: {account_details['owner']}")
                print(f"📊 Tamaño datos: {account_details['data_length']} bytes")

            # ====================== TOKENS ======================
            token_accounts = await account_info.get_token_accounts(address)

            if not token_accounts:
                print("❌ No se encontraron tokens SPL en esta wallet")
                return

            print(f"\n🪙 TOKENS SPL ENCONTRADOS: {len(token_accounts)}")
            print("=" * 60)

            i = 0
            for i, token in enumerate(token_accounts, 1):
                    print(f"\n{i}. Token:")
                    print(f"   📍 Mint: {token['mint']}")
                    print(f"   💰 Balance: {token['balance']:,.{min(token['decimals'], 6)}f}")
                    print(f"   🔢 Decimales: {token['decimals']}")
                    print(f"   🏦 Cuenta: {token['account_address']}")

            if i == 0:
                print("📭 No se encontraron tokens SPL en esta wallet")

await get_account_info()


## 3. Transferencias SOL

Enviar SOL a otras wallets.

### 3.1 Estimar Comisiones de Transferencia

In [ ]:
async def get_transfer_fee_estimate(amount_to_send: float, compute_units_limit: int = 200000, priority_fee_micro_lamports: int = 1):
    """
    Función actualizada para estimar comisiones de transferencia
    
    Args:
        amount_to_send: Cantidad de SOL a enviar
        compute_units_limit: Límite de unidades de cómputo (default: 200,000)
        priority_fee_micro_lamports: Precio por unidad de cómputo en micro-lamports (default: 1)
    """
    WALLET_FILE = "wallets/wallet_pumpfun.json"

    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        await wallet_manager.load_wallet(WALLET_FILE)

        if not wallet_manager.is_wallet_loaded():
            print("❌ No hay wallet cargada")
            return

        async with SolanaTransfer(network="mainnet-beta") as transfer_manager:
            fee_estimate = await transfer_manager.get_transfer_fee_estimate(
                amount_to_send=amount_to_send,
                compute_units_limit=compute_units_limit,
                priority_fee_micro_lamports=priority_fee_micro_lamports
            )

            print(f"\n💸 ESTIMACIÓN DE COMISIONES")
            print("=" * 50)
            print(f"💰 Cantidad a enviar: {fee_estimate['amount_sol']} SOL")
            print(f"💳 Tarifa base: {fee_estimate['base_fee_sol']:.9f} SOL")
            print(f"⚡ Tarifa priorización: {fee_estimate['priority_fee_sol']:.9f} SOL")
            print(f"📊 Tarifa total: {fee_estimate['total_fee_sol']:.9f} SOL")
            print(f"💵 Costo total: {fee_estimate['total_cost_sol']:.9f} SOL")
            print(f"🌐 Red: {fee_estimate['network']}")
            print(f"⚙️ Unidades cómputo: {fee_estimate['compute_units_limit']:,}")
            print(f"🎯 Prioridad: {fee_estimate['priority_fee_micro_lamports']} micro-lamports")
            print(f"🔧 Fuente: {fee_estimate['fee_source']}")

await get_transfer_fee_estimate(0.001)

### 3.2 Transferir SOL

In [ ]:
# ⚠️ CUIDADO: Esta celda transfiere SOL real
# Configura los valores para usar

async def transfer_sol():
    WALLET_FILE = "wallets/wallet_pumpfun.json"
    DESTINATION_ADDRESS = "62bzYQAbEhtc7eP3qb9G557fhLqcfNxeTKr6v1UshnD1"
    AMOUNT_SOL = 0.00555555  # Cantidad en SOL

    async with SolanaWalletManager(network="mainnet-beta") as wallet_manager:
        await wallet_manager.load_wallet(WALLET_FILE)
        if not wallet_manager.is_wallet_loaded():
            print("❌ No hay wallet cargada")
            return

        async with SolanaUtils(network="mainnet-beta") as utils:
            # Validar dirección destino
            if not await utils.validate_address(DESTINATION_ADDRESS):
                print("❌ Dirección destino inválida")
                return

        async with SolanaTransfer(network="mainnet-beta") as transfer_manager:
            print(f"🔄 Iniciando transferencia de {AMOUNT_SOL} SOL...")

            signature = await transfer_manager.transfer_sol(
                from_keypair=wallet_manager.keypair,
                to_address=DESTINATION_ADDRESS,
                amount_sol=AMOUNT_SOL
            )

            if signature:
                print(f"✅ Transferencia exitosa!")
                print(f"🔗 Signature: {signature}")
            else:
                print("❌ Error en la transferencia")

await transfer_sol()

## 4. Utilidades de Solana

Herramientas útiles para validación, conversiones y información de red.

### 4.1 Validar Direcciones

In [ ]:
async def validate_address():
    # Validar direcciones de Solana
    test_addresses = [
        "11111111111111111111111111111112",  # System Program
        "So11111111111111111111111111111111111111112",  # WSOL
        "direccion_invalida",  # Inválida
        "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA"  # Token Program
    ]

    async with SolanaUtils(network="mainnet-beta") as utils:
        for i, address in enumerate(test_addresses, 1):
            print(f"\n{i}. {address[:30]}...")
            is_valid = await utils.validate_address(address)
            print(f"   Resultado: {'✅ Válida' if is_valid else '❌ Inválida'}")

await validate_address()

### 4.2 Información de Red

In [ ]:
async def get_network_info():
    # Obtener información de la red Solana
    async with SolanaUtils(network="mainnet-beta") as utils:
        network_info = await utils.get_network_info()

        if not network_info:
            print("❌ No se pudo obtener información de red")
            return

    print(f"\n🌐 INFORMACIÓN DE RED")
    print("=" * 40)
    print(f"🌍 Red: {network_info['network']}")
    print(f"🔗 RPC URL: {network_info['rpc_url']}")
    print(f"📊 Slot actual: {network_info['current_slot']:,}")
    print(f"✅ Estado: {network_info['status']}")
    print(f"⏰ Timestamp: {network_info['timestamp']}")

await get_network_info()


### 4.3 Conversiones SOL ↔ Lamports

In [ ]:
# Conversiones entre SOL y lamports
async def convert_sol_to_lamports():
    test_values = {
        "sol_amounts": [0.001, 0.1, 1.0, 10.5],
        "lamport_amounts": [1000000, 100000000, 1000000000, 5000000000]
    }

    print("🔄 CONVERSIONES SOL ↔ LAMPORTS")
    print("=" * 50)

    async with SolanaUtils(network="mainnet-beta") as utils:
        print("\n📈 SOL → Lamports:")
        for sol in test_values["sol_amounts"]:
            lamports = await utils.convert_sol_to_lamports(sol)
            print(f"   {sol} SOL = {lamports:,} lamports")

        print("\n📉 Lamports → SOL:")
        for lamports in test_values["lamport_amounts"]:
            sol = await utils.convert_lamports_to_sol(lamports)
            formatted = await utils.format_balance(lamports)
            print(f"   {lamports:,} lamports = {formatted}")

await convert_sol_to_lamports()

### 4.4 Precio de SOL y Cálculos USD

In [ ]:
# Obtener precio actual de SOL y calcular valores
async def get_current_prince_and_calculate_values():
    sol_amounts = [0.1, 1.0, 5.0, 10.0]

    print("💵 PRECIO DE SOL Y VALORES USD")
    print("=" * 50)

    # Obtener precio actual
    async with SolanaUtils(network="mainnet-beta") as utils:
        sol_price = await utils.get_solana_price_usd()

        if not sol_price:
            print("❌ No se pudo obtener el precio de SOL")
            print("💡 Verifica tu conexión a internet")
            return

        print(f"\n💰 Precio actual: ${sol_price:.2f} USD")
        print("\n📊 Valores calculados:")
        
        for amount in sol_amounts:
            value_usd = amount * sol_price
            print(f"   {amount} SOL = ${value_usd:.2f} USD")

await get_current_prince_and_calculate_values()